In [1]:

%pip install pandas numpy scikit-learn

Note: you may need to restart the kernel to use updated packages.


# Load libraries and datasets

In [1]:
import pandas as pd
import numpy as np

In [7]:
# players = pd.read_csv('./raw_datasets/players.csv')
# purchased_games = pd.read_csv("./raw_datasets/purchased_games.csv")
# games = pd.read_csv("./raw_datasets/games.csv")
# history = pd.read_csv("./raw_datasets/history.csv")
prices = pd.read_csv("./raw_datasets/prices.csv")

In [8]:
# print(players.isnull().sum())
# print(games.isnull().sum())
# print(purchased_games.isnull().sum())
# print(history.isnull().sum())
print(prices.isnull().sum())

gameid                 0
usd               907148
eur              1703086
gbp               908184
jpy               919973
rub              1006291
date_acquired          0
dtype: int64


# Clean data

## Remove duplicates

In [ ]:
# players = players.drop_duplicates()
# purchased_games = purchased_games.drop_duplicates()
# games = games.drop_duplicates()
# history = history.drop_duplicates()
# prices = prices.drop_duplicates()

## Standardize date format

In [ ]:
# players['created'] = pd.to_datetime(players['created'])
# games['release_date'] = pd.to_datetime(games['release_date'])
# history['date_acquired'] = pd.to_datetime(history['date_acquired'])
# prices['date_acquired'] = pd.to_datetime(prices['date_acquired'])

# Handle missing data

## players.csv

In [7]:
players.fillna({'country': 'Unknown'}, inplace=True)

players.fillna({'created': players['created'].median()}, inplace=True)

print(players.isnull().sum())

playerid    0
country     0
created     0
dtype: int64


## games.csv

In [ ]:
games.dropna(subset=['title'], inplace=True)
games.dropna(subset=['developers'], inplace=True)
games.dropna(subset=['publishers'], inplace=True)
games.dropna(subset=['genres'], inplace=True)
games.dropna(subset=['supported_languages'], inplace=True)

print(games.isnull().sum())

gameid                 0
title                  0
developers             0
publishers             0
genres                 0
supported_languages    0
release_date           0
dtype: int64


In [15]:
games.to_csv('./clean_datasets/games.csv', index=False)

## purchased_games.csv

In [9]:
purchased_games.fillna({'library': '[]'}, inplace=True)

print(purchased_games.isnull().sum())

playerid    0
library     0
dtype: int64


## history.csv (no null values)

## prices.csv

In [9]:
latest_prices = prices.sort_values("date_acquired").groupby("gameid").last().reset_index()

In [ ]:
# replace gameid with NaN price with 0 (free game or not released yet)
latest_prices.fillna({'usd': 0}, inplace=True)

In [12]:
latest_prices.drop(columns=["eur", "gbp", "jpy", "rub", "date_acquired"], inplace=True)

In [15]:
# change column name
latest_prices.rename(columns={"usd": "price_usd"}, inplace=True)

In [ ]:
# exchange_rates = {
#     "usd": 1.0,
#     "eur": 0.93,
#     "gbp": 0.77,
#     "jpy": 150.75,
#     "rub": 84.51
# }

# def convert_to_usd(row):
#     prices_in_usd = [row[currency] / exchange_rates[currency] for currency in exchange_rates if not pd.isna(row[currency])]
#     return sum(prices_in_usd) / len(prices_in_usd) if prices_in_usd else None

# latest_prices = prices.sort_values("date_acquired").groupby("gameid").last().reset_index()

# # Create a new column with the converted USD price
# latest_prices["price_usd"] = latest_prices.apply(convert_to_usd, axis=1)

# # Drop old currency columns
# latest_prices = latest_prices.drop(columns=["usd", "eur", "gbp", "jpy", "rub"])

In [15]:
purchased_games["library"] = purchased_games["library"].apply(eval)

In [ ]:
achievements = pd.read_csv("./raw_datasets/achievements.csv")

In [23]:
friends = pd.read_csv("./datasets/friends.csv")

In [ ]:
# convert friends column to list of playerid, NaN to empty list
friends["friends"] = friends["friends"].apply(lambda x: eval(x) if not pd.isna(x) else [])

In [27]:
# write dataset back to csv
import os
if not os.path.exists('./clean_datasets'):
    os.makedirs('./clean_datasets')

players.to_csv('./clean_datasets/players.csv', index=False)
games.to_csv('./clean_datasets/games.csv', index=False)
purchased_games.to_csv('./clean_datasets/purchased_games.csv', index=False)
history.to_csv('./clean_datasets/history.csv', index=False)
latest_prices.to_csv('./clean_datasets/prices.csv', index=False)
achievements.to_csv('./clean_datasets/achievements.csv', index=False)
friends.to_csv('./clean_datasets/friends.csv', index=False)